In [1]:
import pandas as pd
import numpy as np
import openpyxl as opxl
import os.path
import pycountry
import pycountry_convert as pc
import datetime as dt
import json
import os.path

def importExcelSheets():
    magento = pd.read_excel('database_sheets/magento_may.xlsx')
    new_magento = pd.read_excel('database_sheets/magento_july.xlsx')
    magento_sept = pd.read_excel('database_sheets/magento_sept.xlsx')
    lot_master = pd.read_excel('database_sheets/lot_master.xlsx', dtype = object)
    prms = pd.read_excel('database_sheets/prms.xlsx')
    unspsc_codes = pd.read_excel('database_sheets/unspsc_codes.xlsx')
    unspsc_codes.columns = unspsc_codes.iloc[0]
    origin = pd.read_excel('database_sheets/country_of_origin.xlsx', dtype=object)
    pd.set_option("max_rows", None)
    pd.set_option("max_columns", None)
    return [magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept]

def tidyDescription(desc):
    desc = desc.replace('&TRADE', '')
    desc = desc.replace('Â®', '')
    desc = desc.replace('Î¼', '')
    desc = desc.replace('& Growâ„¢', '')
    desc = desc.replace('â‰', '')
    desc = desc.replace('&Beta', '')
    desc = desc.replace('Â‰¥', '')
    desc = desc.replace('Â', '')
    desc = desc.replace('<em>', '')
    desc = desc.replace('</em>', '')
    desc = desc.replace('®','')
    desc = desc.replace('<F128>','')
    desc = desc.replace('<130>','')
    desc = desc.replace('<F255>','')
    desc = desc.replace('&deg;', '°')
    desc = desc.replace('deg ', '°')
    if desc.endswith(','):
        desc = desc[:len(desc)-1]
    return desc

database_sheets = importExcelSheets()
categories = pd.read_excel('database_sheets/product_categories.xlsx')

In [114]:
antibodies = pd.read_excel('forms/vwr_enrichment_antibodies.xlsx')
ppe = pd.read_excel('forms/vwr_enrichment_ppe.xlsx')
sera = pd.read_excel('forms/vwr_enrichment_sera.xlsx')
ccm = pd.read_excel('forms/vwr_enrichment_ccm.xlsx')
chemicals = pd.read_excel('forms/vwr_enrichment_chemicals.xlsx')
enrichment = pd.read_excel('forms/vwr_enrichment_form.xlsx', dtype = object)
# antibodies.columns = np.arange(len(antibodies.columns))
# ppe.columns = np.arange(len(ppe.columns))
# sera.columns = np.arange(len(sera.columns))
# ccm.columns = np.arange(len(ccm.columns))
# chemicals.columns = np.arange(len(chemicals.columns))
ccm = columnize(ccm, 7, 6)
sera = columnize(sera, 7, 6)
chemicals.columns = chemicals.iloc[10]
antibodies.columns = antibodies.iloc[1]
ppe.columns = ppe.iloc[0]

In [116]:
sera.iloc[:8]

,Examples below…,VWR-US Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Accessory,Table Heading for Order Tab,Serum Source,Animal Serum Type,Human Serum Type,FBS Origin,Serum Treatment,Disease Type,Serum Format,Sterility,Description,Size,Packaging,Endotoxin,Hemoglobin,Shelf Life,Source,Application,Concentration,Purification,Regulatory Status,Storage Buffer,Storage Temperature,ELISA,Flow Cytometry,ImmunoChemistry,ImmunoFluorescence,ImmunoPrecipitation,Western Blot,Search Keywords,,Brand Name,Supplier Name,Product Title\n(max. 100 characters),Quick Summary Text\n(max. 100 words),Key Features/Benefits,Extended Exposition Text,Certification Information\n(max. 200 characters),Accessory Text\n(max. 300 characters),Ordering Information,Delivery Information,Packaging Information,Caution Statements,,Y/N,Y/N,Y/N,Y/N,Y/N,Y/N,Y/N,Y/N,"ex. ENERGY STAR, FSC, Ecologo, etc.",Include EPP Narrative
0,-Columns highlighted in Yellow are required.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To the left is part number specific informatio...,NaN,NaN,NaN,NaN,To the right is the parent/product information...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-Add new columns for each Order Tab and Spe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-At least one Order Tab Attribute is needed...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-The Parent/Product information only needs ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A unique value to group part numbers together ...,Examples:\nUS: 12345-678\nSciEd: 470123-456,Examples:\nEU MEL: PALL123456\nEU ECN: 123-4567,Examples:\nCA: CA12345-678,NaN,Indicate 'Y' value for if the SKU is an Access...,If needed:\nAdd a sub-heading to break part nu...,Human or Animal,e.g. Fetal Bovine Serum; Equine Serum; etc.,e.g. OTC; AB; etc.,For Fetal Bovine Serum only:\nUnited States; A...,e.g. Irradiated; Heat Inactivated; etc.,For Human Serum only:\nList the disease type,Choose from one of the following: \nLyophilize...,Sterile or Nonsterile,Fill these attributes in as needed,NaN,NaN,NaN,NaN,NaN,Fill this in if you need to specify the Serum ...,Fill these attributes in as needed,NaN,NaN,NaN,NaN,NaN,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Add terms here that customers will use to find...,NaN,"The Brand Name, Supplier Name and Product Titl...",NaN,NaN,"In one statement, state why a customer might b...",Feature/benefit statements.\n3-7 bullet points...,Write text paragraphs about the product.\nSepa...,Add statements of certification here.\n\nOnly ...,Describe any accessory items featured on the p...,Add text specific to the ordering of the produ...,Add text specific to the delivery of the produ...,Add text specific to the packaging of the prod...,Add any Caution or Warning statements here.\n\...,NaN,If this is an Environmentally Preferable Produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Please explain your EPP claim below (ie: speci...,Please explain your EPP claim below (ie: speci...
6,Group Key,VWR-US Cat. No

In [113]:
sera.iloc[:8]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,-Columns highlighted in Yellow are required.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To the left is part number specific informatio...,NaN,NaN,NaN,NaN,To the right is the parent/product information...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-Add new columns for each Order Tab and Spe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-At least one Order Tab Attribute is needed...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-The Parent/Product information only needs ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A unique value to group part numbers together ...,Examples:\nUS: 12345-678\nSciEd: 470123-456,Examples:\nEU MEL: PALL123456\nEU ECN: 123-4567,Examples:\nCA: CA12345-678,NaN,Indicate 'Y' value for if the SKU is an Access...,If needed:\nAdd a sub-heading to break part nu...,Human or Animal,e.g. Fetal Bovine Serum; Equine Serum; etc.,e.g. OTC; AB; etc.,For Fetal Bovine Serum only:\nUnited States; A...,e.g. Irradiated; Heat Inactivated; etc.,For Human Serum only:\nList the disease type,Choose from one of the following: \nLyophilize...,Sterile or Nonsterile,Fill these attributes in as needed,NaN,NaN,NaN,NaN,NaN,Fill this in if you need to specify the Serum ...,Fill these attributes in as needed,NaN,NaN,NaN,NaN,NaN,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Enter Yes if applicable,Add terms here that customers will use to find...,NaN,"The Brand Name, Supplier Name and Product Titl...",NaN,NaN,"In one statement, state why a customer might b...",Feature/benefit statements.\n3-7 bullet points...,Write text paragraphs about the product.\nSepa...,Add statements of certification here.\n\nOnly ...,Describe any accessory items featured on the p...,Add text specific to the ordering of the produ...,Add text specific to the delivery of the produ...,Add text specific to the packaging of the prod...,Add any Caution or Warning statements here.\n\...,NaN,If this is an Environmentally Preferable Produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Please explain your EPP claim below (ie: speci...,Please explain your EPP claim below (ie: speci...
6,Group Key,VWR-US Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Accessory,Table Heading for Order Tab,Mandatory Attribute,Mandatory Attribute,Mandatory Attribute,Mandatory Attribute,Mandatory Attribute,Mandatory Attribute,Order Tab Attribute,Order Tab Attribute,Additional Order Tab Attribute,Additional Order Tab Attribute,Additional Order Tab Attribute,Additional Order Tab Attribute,Additional Order Tab Attribute,Additional Order Tab Attribute,Additional Order Tab Attribute,Additional Specification Tab Attribute,Additional Specification Tab Attribute,Additional Specification Tab Attribute,Additional Specification Tab Attribute,Additional Specification Tab Attribute,Additional Specification Tab Attribute,Additional Specification Tab Attribu

In [197]:
def fillVWR_Enrichment_Antibodies(enrichment, magento, prms):
    for i in range(2, len(enrichment)):
        sku = enrichment['Supplier Part No.'][i]
        product_info = magento.loc[magento['sku'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        
        if not prms_info.empty:
            ship_temp = prms_info['Ship Temp'].values[0]
            storage_temp = prms_info['Storage Temp'].values[0]
            
            if ship_temp == 'CP':
                enrichment['Shipping Temperature'][i] = '2C to 8C'
            elif ship_temp == 'DI':
                enrichment['Shipping Temperature'][i] = '-30C to -2C'
            elif ship_temp == 'AM':
                enrichment['Shipping Temperature'][i] = '15C to 30C'
                
            if storage_temp == 'AM':
                enrichment['Storage Temperature'][i] = '15C to 30C'
            elif storage_temp == 'FR':
                enrichment['Storage Temperature'][i] = '-30C to -2C'
            elif storage_temp == 'RF':
                enrichment['Storage Temperature'][i] = '2C to 8C'
            elif storage_temp == '70' or storage_temp == '80':
                enrichment['Storage Temperature'][i] = '-70C'
        
        if not product_info.empty:
            name = product_info['name'].values[0]
            # description = product_info['description'].values[0]
            # short_desc = product_info['short_description'].values[0]
            pack_size_joined = product_info['pack_size_joined'].values[0]
            antibody_type = product_info['antibody_type'].values[0]
            host = product_info['host'].values[0]
            conjugate = product_info['conjugate'].values[0]
            clone_name = product_info['clone_name'].values[0]
            concentration = product_info['concentration'].values[0]
            immunogen = product_info['immunogen'].values[0]
            molecular_weight = product_info['molecular_weight'].values[0]
            purity = product_info['purity'].values[0]
            # application_notes = product_info['application_notes'].values[0]

            # enrichment['Title / Short Description / Antibody Name'][i] = description

            # if type(short_desc) == str and len(short_desc) > 30:
            #     enrichment['Short Description'][i] = short_desc[:30]
            # else:
            #     enrichment['Short Description'][i] = short_desc

            # enrichment['Long Text Description'][i] = description
            enrichment['Size (with unit)'][i] = pack_size_joined

            if type(antibody_type) == str:
                enrichment['Type'][i] = str(antibody_type).replace(' Antibody', '')
                enrichment['Clonality'][i] = str(antibody_type).replace(' Antibody', '')

            enrichment['Host'][i] = host
            enrichment['Conjugation'][i] = conjugate
            
            if type(clone_name) == str and len(clone_name) > 0:
                enrichment['Clone Name/Number'][i] = 'Clone: ' + clone_name
                
            enrichment['Reactivity'][i] = host
            enrichment['Concentration'][i] = concentration
            enrichment['Immunogen'][i] = immunogen
            enrichment['Molecular Weight'][i] = molecular_weight
            enrichment['Purification Method'][i] = purity

            if 'ELISA' in name or 'elisa' in name:
                enrichment['ELISA'][i] = 'Yes'
            else:
                enrichment['ELISA'][i] = 'No'

            # enrichment['Application Notes'][i] = application_notes

    new_enrichment = opxl.load_workbook('forms/vwr_enrichment_antibodies.xlsx')
    sheet = new_enrichment.active
    if sheet.max_row < len(enrichment):
        sheet.append([''])
        for j in range(len(enrichment) - sheet.max_row + 4):
            sheet.insert_rows(sheet.max_row)
    i = 2
    for row in sheet.iter_rows(min_row=4):
        if i < (len(enrichment)):
            for j in range(len(enrichment.columns)):
                row[j].value = enrichment.iloc[i, j]
        else:
            break
        i += 1
            
    new_enrichment.save('outputs/enrichment_outputs/vwr_enrichment_antibody_output.xlsx')

def fillVWR_Enrichmnent_PPE(enrichment, magento, prms):
    for i in range(2, len(enrichment)):
        sku = enrichment['Supplier Part No.'][i]
        product_info = magento.loc[magento['sku'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        
        if not prms_info.empty:
            ship_temp = prms_info['Ship Temp'].values[0]
            storage_temp = prms_info['Storage Temp'].values[0]
            
            if ship_temp == 'CP':
                enrichment['Shipping temperature'][i] = '2C to 8C'
            elif ship_temp == 'DI':
                enrichment['Shipping temperature'][i] = '-30C to -2C'
            elif ship_temp == 'AM':
                enrichment['Shipping temperature'][i] = '15C to 30C'
                
            if storage_temp == 'AM':
                enrichment['Storage conditions'][i] = '15C to 30C'
            elif storage_temp == 'FR':
                enrichment['Storage conditions'][i] = '-30C to -2C'
            elif storage_temp == 'RF':
                enrichment['Storage conditions'][i] = '2C to 8C'
            elif storage_temp == '70' or storage_temp == '80':
                enrichment['Storage conditions'][i] = '-70C'
        
        if not product_info.empty:
            name = product_info['name'].values[0]
            description = product_info['description'].values[0]
            # short_desc = product_info['short_description'].values[0]
            pack_size_joined = product_info['pack_size_joined'].values[0]
            # antibody_type = product_info['antibody_type'].values[0]
            host = product_info['host'].values[0]
            conjugate = product_info['conjugate'].values[0]
            # clone_name = product_info['clone_name'].values[0]
            concentration = product_info['concentration'].values[0]
            # immunogen = product_info['immunogen'].values[0]
            molecular_weight = product_info['molecular_weight'].values[0]
            purity = product_info['purity'].values[0]
            # application_notes = product_info['application_notes'].values[0]
            cas_number = product_info['cas_number'].values[0]
            
            # if type(short_desc) == str:
            #     short_desc = tidyDescription(short_desc)
            #     short_desc = ''.join([i for i in short_desc if i.isalnum() or i == ' '])
            #     if len(short_desc) > 30:
            #         enrichment['short Description'][i] = short_desc[:30]
            #     else:
            #         enrichment['short Description'][i] = short_desc
                    
            # if type(description) == str:
            #     description = tidyDescription(description)
            #     description = ''.join([i for i in description if i.isalnum() or i == ' '])
            #     enrichment['long Description'][i] = description
                
            if 'recombinant' in description or 'Recombinant' in description:
                enrichment['Protein/Peptide/Enzyme type'][i] = 'recombinant'

            if type(name) == str:
                enrichment['Product Title'][i] = name + ' MP Biomedical'
                if 'protein' in name:
                    enrichment['Product class'][i] = 'protein'
                elif 'enzyme' in name:
                    enrichment['Product class'][i] = 'enzyme'
                elif 'peptide' in name:
                    enrichment['Product class'][i] = 'peptide'
                
            enrichment['Size (with unit)'][i] = pack_size_joined
                
            enrichment['Species'][i] = host
            enrichment['Conjugation'][i] = conjugate
            enrichment['CAS#'][i] = cas_number
            enrichment['Purity'][i] = purity
            enrichment['Molecular Weight'][i] = molecular_weight
            enrichment['Concentration'][i] = concentration
            
    new_enrichment = opxl.load_workbook('forms/vwr_enrichment_ppe.xlsx')
    sheet = new_enrichment.active
    if sheet.max_row < len(enrichment):
        sheet.append([''])
        for j in range(len(enrichment) - sheet.max_row + 4):
            sheet.insert_rows(sheet.max_row)
    i = 2
    for row in sheet.iter_rows(min_row=4):
        if i < (len(enrichment)):
            for j in range(len(enrichment.columns)):
                row[j].value = enrichment.iloc[i, j]
        else:
            break
        i += 1

    new_enrichment.save('outputs/enrichment_outputs/vwr_enrichment_ppe_output.xlsx')

    return enrichment

def fillVWR_Enrichmnent_Sera(enrichment, magento):

    for i in range(11, len(enrichment)):
        sku = enrichment['Supplier Part No.'][i]
        product_info = magento.loc[magento['sku'] == sku]
        
        if not product_info.empty:
            pack_size_joined = product_info['pack_size_joined'].values[0]
            host = product_info['host'].values[0]
            concentration = product_info['concentration'].values[0]
            purity = product_info['purity'].values[0]
            application_notes = product_info['application_notes'].values[0]
            sterility = product_info['sterility'].values[0]
            keywords = product_info['keywords'].values[0]
            
            # if host == 'human' or host == 'Human':
            #     enrichment.loc[i, 'Serum Source'] = 'Human'
            # elif type(host) == str and len(host) > 0:
            #     enrichment.loc[i, 'Serum Source'] = 'Animal'
            if type(host) == str and len(host) > 0 and host != 'human' or host != "Human":
                enrichment.loc[i, 'Animal Serum Type'] = host
            
            enrichment.loc[i, 'Sterility'] = sterility
            enrichment.loc[i, 'Size'] = pack_size_joined
            enrichment.loc[i, 'Application'] = application_notes
            enrichment.loc[i, 'Concentration'] = concentration
            enrichment.loc[i, 'Purification'] = purity
            
            if type(keywords) == str:
                keywords = keywords.replace(', ', ';')
                keywords = keywords.replace(',', ';')
                keywords = keywords.replace(' |', ';')
                keywords = keywords.replace('| ', ';')
                keywords = keywords.replace(' | ', ';')
                keywords = keywords.replace('|', ';')
                enrichment.loc[i, 'Search Keywords'] = keywords
                
    new_enrichment = opxl.load_workbook('forms/vwr_enrichment_sera.xlsx')
    sheet = new_enrichment.active
    if sheet.max_row < len(enrichment):
        sheet.append([''])
        for j in range(len(enrichment) - sheet.max_row + 12):
            sheet.insert_rows(sheet.max_row)
    i = 11
    for row in sheet.iter_rows(min_row=13):
        if i < (len(enrichment)):
            for j in range(len(enrichment.columns)):
                row[j].value = enrichment.iloc[i, j]
        else:
            break
        i += 1

    new_enrichment.save('outputs/enrichment_outputs/vwr_enrichment_sera_output.xlsx')

    return enrichment

def fillVWR_Enrichmnent_CCM(enrichment, magento, prms):
    for i in range(11, len(enrichment)):
        sku = enrichment.loc[i, 'Supplier Part No.']
        product_info = magento.loc[magento['sku'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        
        if not prms_info.empty:
            # ship_temp = prms_info['Ship Temp'].values[0]
            storage_temp = prms_info['Storage Temp'].values[0]
                
            if storage_temp == 'AM':
                enrichment.loc[i, 'Storage and stability'] = '15C to 30C'
            elif storage_temp == 'FR':
                enrichment.loc[i, 'Storage and stability'] = '-30C to -2C'
            elif storage_temp == 'RF':
                enrichment.loc[i, 'Storage and stability'] = '2C to 8C'
            elif storage_temp == '70' or storage_temp == '80':
                enrichment.loc[i, 'Storage and stability'] = '-70C'
        
        if not product_info.empty:
            name = product_info['name'].values[0]
            description = product_info['description'].values[0]
            short_desc = product_info['short_description'].values[0]
            pack_size_joined = product_info['pack_size_joined'].values[0]
            # host = product_info['host'].values[0]
            # concentration = product_info['concentration'].values[0]
            # purity = product_info['purity'].values[0]
            application_notes = product_info['application_notes'].values[0]
            # sterility = product_info['sterility'].values[0]
            # keywords = product_info['keywords'].values[0]
            culture_media_type = product_info['culture_media_type'].values[0]
            formulation = product_info['formulation'].values[0]

            
            enrichment.loc[i, 'Cell Culture Media Type'] = culture_media_type
            enrichment.loc[i, 'Media Formulation'] = formulation
            enrichment.loc[i, 'Media Format'] = formulation
            enrichment.loc[i, 'Size'] = pack_size_joined
            enrichment.loc[i, 'Application'] = application_notes
            
            if type(short_desc) == str:
                short_desc = tidyDescription(short_desc)
                enrichment.loc[i, 'Description'] = short_desc
                enrichment.loc[i, 'Quick Summary Text\n(max. 100 words)'] = short_desc
            
            enrichment.loc[i, 'Brand Name'] = 'MP Bio'
            enrichment.loc[i, 'Product Title\n(max. 100 characters)'] = name
            
            if type(description) == str:
                description = tidyDescription(description)
                enrichment.loc[i, 'Extended Exposition Text'] = description
    new_enrichment = opxl.load_workbook('forms/vwr_enrichment_ccm.xlsx')
    sheet = new_enrichment.active
    if sheet.max_row < len(enrichment):
        sheet.append([''])
        for j in range(len(enrichment) - sheet.max_row + 12):
            sheet.insert_rows(sheet.max_row)
    i = 11
    for row in sheet.iter_rows(min_row=13):
        if i < (len(enrichment)):
            for j in range(len(enrichment.columns)):
                row[j].value = enrichment.iloc[i, j]
        else:
            break
        i += 1

    new_enrichment.save('outputs/enrichment_outputs/vwr_enrichment_ccm_output.xlsx')

    return enrichment

def fillVWR_Enrichmnent_Chemicals(enrichment, magento, prms):

    for i in range(17, len(enrichment)):
        sku = enrichment['Supplier Part No.'][i]
        product_info = magento.loc[magento['sku'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        # pubchem_data = getPubchemData(sku, magento)
        pubchem_data = None #change this

        if not pubchem_data == None:
            density = pubchem_data['Density']
            boiling_point = pubchem_data['Boiling Point']

            if (not density == None) and len(density) > 0 and (not 'None' in density):
                enrichment.loc[i, 'Density'] = density + " | pulled from Pubchem"
            if (not boiling_point == None) and len(boiling_point) > 0 and (not 'None' in boiling_point):
                boiling_point = tidyDescription(boiling_point)
                enrichment.loc[i, 'Boiling Point'] = boiling_point + " | pulled from Pubchem"
        
        if not prms_info.empty:
            pack_size = prms_info['Pack Size'].values[0]
            storage_temp = prms_info['Storage Temp'].values[0]
    
            enrichment.loc[i, 'Size'] = pack_size
        
            if storage_temp == 'AM':
                enrichment.loc[i, 'Storage Temperature'] = 'Ambient'
            elif storage_temp == 'FR':
                enrichment.loc[i, 'Storage Temperature'] = 'Dry Ice'
            elif storage_temp == 'RF':
                enrichment.loc[i, 'Storage Temperature'] = 'Cold Pack'
            elif storage_temp == '70' or storage_temp == '80':
                enrichment.loc[i, 'Storage Temperature'] = 'Dry Ice'
        
        if not product_info.empty:
            name = product_info['name'].values[0]
            description = product_info['description'].values[0]
            keywords = product_info['keywords'].values[0]
            cas_number = product_info['cas_number'].values[0]
            melting_point = product_info['melting_point'].values[0]
            if type(melting_point) == str:
                melting_point = tidyDescription(melting_point)

            if type(keywords) == str:
                keywords = keywords.replace(', ', ';')
                keywords = keywords.replace(',', ';')
                keywords = keywords.replace(' |', ';')
                keywords = keywords.replace('| ', ';')
                keywords = keywords.replace(' | ', ';')
                keywords = keywords.replace('|', ';')
                enrichment.loc[i, 'Search Keywords'] = keywords
            
            if type(name) == str:
                name = tidyDescription(name)
                name = ''.join([i for i in name if i.isalnum() or i == ' '])
                enrichment.loc[i, 'Product Name'] = name
                
            if type(description) == str:
                description = tidyDescription(description)
                description = ''.join([i for i in description if i.isalnum() or i == ' '])
                enrichment.loc[i, 'Product Text'] = description
                
            enrichment.loc[i, 'CAS'] = cas_number
            enrichment.loc[i, 'Melting Point'] = melting_point
    new_enrichment = opxl.load_workbook('forms/vwr_enrichment_chemicals.xlsx')
    sheet = new_enrichment.active
    if sheet.max_row < len(enrichment):
        sheet.append([''])
        for j in range(len(enrichment) - sheet.max_row + 18):
            sheet.insert_rows(sheet.max_row)
    i = 17
    for row in sheet.iter_rows(min_row=19):
        if i < (len(enrichment)):
            for j in range(len(enrichment.columns)):
                row[j].value = enrichment.iloc[i, j]
        else:
            break
        i += 1

    new_enrichment.save('outputs/enrichment_outputs/vwr_enrichment_chemicals_output.xlsx')

    return enrichment

def fillVWR_Enrichment(filename, magento):
    enrichment = pd.read_excel('forms/vwr_enrichment_form.xlsx', dtype = object)
    # enrichment.columns = np.arange(len(enrichment.columns))
    enrichment.columns = enrichment.iloc[6]
    new_columns = [i.strip() if type(i) == str else i for i in enrichment.columns]
    enrichment.columns = new_columns
    wb = opxl.load_workbook(filename)
    skus = wb.active
    for i in range(2, skus.max_row+1):
        enrichment.loc[i+9, 'Supplier Part No.'] = str(skus['A'+str(i)].value)
        
    for i in range(11, skus.max_row+10):
        sku = enrichment['Supplier Part No.'][i]
        product_info = magento.loc[magento['sku'] == sku]
        
        if not product_info.empty:
            short_desc = product_info['short_description'].values[0]
            keywords = product_info['meta_keywords'].values[0]
            name = product_info['name'].values[0].upper()
            description = product_info['description'].values[0]
            
            enrichment.loc[i, 'Brand Name'] = 'MP Biomedical'
            enrichment.loc[i, 'Supplier Name'] = 'MP Biomedical'
            enrichment.loc[i, 'Product Title\n(max. 100 characters)'] = tidyDescription(str(name))
            enrichment.loc[i, 'Quick Summary Text\n(max. 100 words)'] = tidyDescription(str(short_desc))
            enrichment.loc[i, 'Key Features/Benefits'] = tidyDescription(str(description))
            enrichment.loc[i, 'Extended Exposition Text'] = keywords
            
    new_enrichment = opxl.load_workbook('forms/vwr_enrichment_form.xlsx')
    regulatory_sheet = new_enrichment.active
    if regulatory_sheet.max_row < skus.max_row:
        regulatory_sheet.append([''])
        for j in range(skus.max_row - regulatory_sheet.max_row + 11):
            regulatory_sheet.insert_rows(regulatory_sheet.max_row)
    i = 11
    for row in regulatory_sheet.iter_rows(min_row=13):
        if i < (skus.max_row+11):
            for j in range(len(enrichment.columns)):
                row[j].value = enrichment.iloc[i, j]
        else:
            break
        i = i+1

    new_enrichment.save('outputs/enrichment_outputs/vwr_enrichment_output.xlsx')
    # new_enrichment.save('../../outputs/enrichment_outputs/vwr_enrichment_output.xlsx')


In [189]:
antibodies = pd.read_excel('forms/vwr_enrichment_antibodies.xlsx')
ppe = pd.read_excel('forms/vwr_enrichment_ppe.xlsx')
sera = pd.read_excel('forms/vwr_enrichment_sera.xlsx')
ccm = pd.read_excel('forms/vwr_enrichment_ccm.xlsx')
chemicals = pd.read_excel('forms/vwr_enrichment_chemicals.xlsx')
enrichment = pd.read_excel('forms/vwr_enrichment_form.xlsx', dtype = object)
# ccm.columns = ccm.iloc[6]
# antibodies.columns = np.arange(len(antibodies.columns))
# ppe.columns = np.arange(len(ppe.columns))
# sera.columns = np.arange(len(sera.columns))
# ccm.columns = np.arange(len(ccm.columns))
# chemicals.columns = np.arange(len(chemicals.columns))
enrichment.columns = np.arange(len(enrichment.columns))
# enrichment.columns = enrichment.iloc[6]
# ccm = columnize(ccm, 7, 6)
# sera = columnize(sera, 7, 6)
# chemicals.columns = chemicals.iloc[10]
# antibodies.columns = antibodies.iloc[1]
# ppe.columns = ppe.iloc[0]

In [191]:
enrichment.iloc[[5, 6]]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
5,A unique value to group part numbers together ...,Examples:\nUS: 12345-678\nSciEd: 470123-456,Examples:\nEU MEL: PALL123456\nEU ECN: 123-4567,Examples:\nCA: CA12345-678,NaN,Indicate 'Y' value for if the SKU is an Access...,If needed:\nAdd a sub-heading to break part nu...,Add Attribute Names to Row 9; be specific abou...,NaN,NaN,NaN,NaN,NaN,Add only terms which are related to your speci...,NaN,NaN,"The Brand Name, Supplier Name and Product Titl...",NaN,NaN,"In one statement, state why a customer might b...",Feature/benefit statements.\n3-7 bullet points...,Write text paragraphs about the product.\nSepa...,Add statements of certification here.\n\nOnly ...,Describe any accessory items featured on the p...,Add text specific to the ordering of the produ...,Add text specific to the delivery of the produ...,Add text specific to the packaging of the prod...,Add any Caution or Warning statements here.\n\...,NaN,If this is an Environmentally Preferable Produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Please explain your EPP claim below (ie: speci...,Please explain your EPP claim below (ie: speci...
6,Group Key,VWR-US Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Accessory,Table Heading for Order Tab,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Search Keywords,NaN,NaN,Brand Name,Supplier Name,Product Title\n(max. 100 characters),Quick Summary Text\n(max. 100 words),Key Features/Benefits,Extended Exposition Text,Certification Information\n(max. 200 characters),Accessory Text\n(max. 300 characters),Ordering Information,Delivery Information,Packaging Information,Caution Statements,NaN,Energy Efficient,Water Efficient,Waste Reducing,Sustainable Materials,Low Manufacturing Impact,Safer Human and Environmental Health,Sustainable Packaging,Product Transparency and Disclosure,Acceptable Verification Method,Additional Product Data/Information


In [186]:
enrichment

6,Group Key,VWR-US Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Accessory,Table Heading for Order Tab,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Search Keywords,NaN,NaN,Brand Name,Supplier Name,Product Title\n(max. 100 characters),Quick Summary Text\n(max. 100 words),Key Features/Benefits,Extended Exposition Text,Certification Information\n(max. 200 characters),Accessory Text\n(max. 300 characters),Ordering Information,Delivery Information,Packaging Information,Caution Statements,NaN,Energy Efficient,Water Efficient,Waste Reducing,Sustainable Materials,Low Manufacturing Impact,Safer Human and Environmental Health,Sustainable Packaging,Product Transparency and Disclosure,Acceptable Verification Method,Additional Product Data/Information
0,-Columns highlighted in Yellow are required.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To the left is part number specific informatio...,NaN,NaN,NaN,NaN,To the right is the parent/product information...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-Add new columns for each Order Tab and Spe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-At least one Order Tab Attribute is needed...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-The Parent/Product information only needs ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A unique value to group part numbers together ...,Examples:\nUS: 12345-678\nSciEd: 470123-456,Examples:\nEU MEL: PALL123456\nEU ECN: 123-4567,Examples:\nCA: CA12345-678,NaN,Indicate 'Y' value for if the SKU is an Access...,If needed:\nAdd a sub-heading to break part nu...,Add Attribute Names to Row 9; be specific abou...,NaN,NaN,NaN,NaN,NaN,Add only terms which are related to your speci...,NaN,NaN,"The Brand Name, Supplier Name and Product Titl...",NaN,NaN,"In one statement, state why a customer might b...",Feature/benefit statements.\n3-7 bullet points...,Write text paragraphs about the product.\nSepa...,Add statements of certification here.\n\nOnly ...,Describe any accessory items featured on the p...,Add text specific to the ordering of the produ...,Add text specific to the delivery of the produ...,Add text specific to the packaging of the prod...,Add any Caution or Warning statements here.\n\...,NaN,If this is an Environmentally Preferable Produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Please explain your EPP claim below (ie: speci...,Please explain your EPP claim below (ie: speci...
6,Group Key,VWR-US Cat. No.,VWR-EU Cat. No.,VWR-Canada Cat. No.,Supplier Part No.,Accessory,Table Heading for Order Tab,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Specification Data,Search Keywords,NaN,NaN,Brand Name,Supplier Name,Product Title\n(max. 100 characters),Quick Summary Text\n(max. 100 words),Key Features/Benefits,Extended Exposition Text,Certification Information\n(max. 200 characters),Accessory Text\n(max. 300 characters),Ordering Information,Delivery Information,Packaging Information,Caution Statements,NaN,Energy Efficient,Water Efficient,Waste Reducing,Sustainable Materials,Low Manufacturing Impact,Safer Human and Environmental Health,Sustainable Packaging,Product Transparency and Disclosure,Acceptable Verification Method,Additional Product Data/Information
7,Examples below…,NaN,NaN,NaN,NaN,NaN,NaN,ex. Description,ex. Capacity,ex. Length,ex. Electrical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y/N,Y/N,Y/N,

In [193]:
def catSheetVWR(sheet_name, cell_str):
    if cell_str == '':
        cell_str = sheet_name
    else:
        cell_str += ', ' + sheet_name

    return cell_str

def columnize(df, primary_row, secondary_row):
    temp = df.fillna('')
    columns = []
    for i in range(len(temp.columns)):
        if temp.iloc[primary_row, i] == '':
            columns.append(temp.iloc[secondary_row, i])
        else:
            columns.append(temp.iloc[primary_row, i])
    df.columns = columns
    return df

def VWREnrichmentDriver(filename, magento, prms, magento_may, categories):
    wb = opxl.load_workbook(filename)
    skus = wb.active
    antibodies = pd.read_excel('forms/vwr_enrichment_antibodies.xlsx', dtype=str)
    ppe = pd.read_excel('forms/vwr_enrichment_ppe.xlsx', dtype=str)
    sera = pd.read_excel('forms/vwr_enrichment_sera.xlsx', dtype=str)
    ccm = pd.read_excel('forms/vwr_enrichment_ccm.xlsx', dtype=str)
    chemicals = pd.read_excel('forms/vwr_enrichment_chemicals.xlsx', dtype=str)
    # antibodies.columns = np.arange(len(antibodies.columns))
    # ppe.columns = np.arange(len(ppe.columns))
    # sera.columns = np.arange(len(sera.columns))
    # ccm.columns = np.arange(len(ccm.columns))
    # chemicals.columns = np.arange(len(chemicals.columns))
    ccm = columnize(ccm, 7, 6)
    sera = columnize(sera, 7, 6)
    chemicals.columns = chemicals.iloc[10]
    antibodies.columns = antibodies.iloc[1]
    ppe.columns = ppe.iloc[0]
    new_columns = [i.strip() if type(i) == str else i for i in ccm.columns]
    ccm.columns = new_columns
    new_columns = [i.strip() if type(i) == str else i for i in sera.columns]
    sera.columns = new_columns
    new_columns = [i.strip() if type(i) == str else i for i in chemicals.columns]
    chemicals.columns = new_columns
    new_columns = [i.strip() if type(i) == str else i for i in antibodies.columns]
    antibodies.columns = new_columns
    new_columns = [i.strip() if type(i) == str else i for i in ppe.columns]
    ppe.columns = new_columns

    categories.columns = categories.iloc[0]
    
    num_antibodies = 0
    num_ppe = 0
    num_sera = 0
    num_ccm = 0
    num_chemicals = 0
    
    for i in range(2, skus.max_row+1):
        sku = str(skus['A'+str(i)].value)
        product_info = magento.loc[magento['sku'] == sku]
        category_info = categories.loc[categories['SKU'] == sku]

        if not category_info.empty:
            category = category_info['Class Name'].values[0]
        else:
            category = ''

        if not product_info.empty:
            name = product_info['name'].values[0]
            cas_number = product_info['cas_number'].values[0]
        else:
            name = ''
            cas_number = ''

        skus['B'+str(i)].value = ''
        if sku.startswith('11'):
            if (category == 'MEDIA' or sku.startswith('1130') or sku.startswith('1131') or sku.startswith('1133') or sku.startswith('1140') or sku.startswith('1141') or sku.startswith('1144') or sku.startswith('1145') or sku.startswith('1148') or sku.startswith('1151')):
                skus['B'+str(i)].value = catSheetVWR('Cell Culture Media', skus['B'+str(i)].value)
                ccm.loc[num_ccm+11, 'Supplier Part No.'] = sku
                num_ccm = num_ccm + 1
            if category == 'BIOCHEMICALS':
                skus['B'+str(i)].value = catSheetVWR('Chemicals', skus['B'+str(i)].value)
                chemicals.loc[num_chemicals+17, 'Supplier Part No.'] = sku
                num_chemicals = num_chemicals + 1
            # if category == 'PCR' or sku.startswith('11EB') or sku.startswith('11EP') or sku.startswith('11MSTP') or sku.startswith('11RTO') or sku.startswith('1199'):

            
        else:
            if (sku.startswith('08') or 'anti-' in name or 'Anti-' in name or 'antibody' in name or 'Antibody' in name) and not sku.startswith('07'):
                skus['B'+str(i)].value = catSheetVWR('Antibodies', skus['B'+str(i)].value)
                antibodies.loc[num_antibodies+2, 'Supplier Part No.'] = sku
                num_antibodies = num_antibodies + 1
            elif 'serum' in name or 'Serum' in name:
                    skus['B'+str(i)].value = catSheetVWR('Sera', skus['B'+str(i)].value)
                    sera.loc[num_sera+11, 'Supplier Part No.'] = sku
                    num_sera = num_sera + 1
            if sku.startswith('02') or 'ase' in name:
                if not (sku.startswith('02') and ('ChLiA' in name or 'peptide' in name or 'Peptide' in name)):
                    skus['B'+str(i)].value = catSheetVWR('Protein, Peptides, Enzymes', skus['B'+str(i)].value)
                    ppe.loc[num_ppe+2, 'Supplier Part No.'] = sku
                    num_ppe = num_ppe + 1
            if 'media' in name or 'Media' in name or 'medium' in name or 'Medium' in name or 'RPMI' in name or (sku.startswith('09') and not 'serum' in name):
                skus['B'+str(i)].value = catSheetVWR('Cell Culture Media', skus['B'+str(i)].value)
                ccm.loc[num_ccm+11, 'Supplier Part No.'] = sku
                num_ccm = num_ccm + 1
            if (type(cas_number) == str and len(cas_number) > 0 and not cas_number == 'Not applicable') or sku.startswith('02'):
                skus['B'+str(i)].value = catSheetVWR('Chemicals', skus['B'+str(i)].value)
                chemicals.loc[num_chemicals+17, 'Supplier Part No.'] = sku
                num_chemicals = num_chemicals + 1

    wb.save(filename)

    fillVWR_Enrichment_Antibodies(antibodies, magento, prms)
    fillVWR_Enrichmnent_PPE(ppe, magento, prms)
    fillVWR_Enrichmnent_Sera(sera, magento)
    # fillVWR_Enrichmnent_CCM(ccm, magento, prms)
    fillVWR_Enrichmnent_Chemicals(chemicals, magento, prms)
    fillVWR_Enrichment(filename, magento_may)

In [52]:
antibodies

1,Blue = Provide All Available\nGreen = Mandatory,Group Key,VWR US Cat. No.,VWR CA Cat. No.,VWR EU Part No.,Supplier Part No.,Title / Short Description / Antibody Name,Short Description,Long Text Description,Brand Name,Size (with unit),Packaging Format,No. of Preparations (if Kit),Type,Host,Antigen\nSymbol,Conjugation,Clonality,Antigen Name,Antigen Synonyms,Gene ID,Clone Name/Number,Isotype,Reactivity,Modification,Epitope,Concentration,Immunogen,Molecular Weight,Amino Acid Number,Purification Method,Sequence,Form,Antibody Fragment,Storage Buffer,Storage Temperature,Shipping Temperature,Format,Cross Adsorption,ELISA,Western Blot,Immunochemistry,Immunoprecipitation,Immunofluorescence,Gel Shift,CHiP,Neutralization,Flow Cytometry,Application Notes,Other Applications,URL ; Supplier Product Page for verification
0,Example Values:,Enter the same value for each part no. that be...,(ex. 12345-678),(ex. CA12345-678),(ex. ABCD12345; prefix & supplier part no.),12345-50,300 Character Limit,30 Character Limit,2000 Character Limit,Campath; Herceptin; etc.,100 mg\n500 µg\n50 mL\n10 Tests,Glass Ampoule,100 Preps,"Primary, Secondary, Control, Tag","Rabbit, Mouse,\nRat,\netc.",IRAK\nAURKC\nIgG F(ab')\netc,Horseradish Peroxidase IR700\n Unconjugated\netc,Monoclonal/Polyclonal,"Alpha-2-macroglobulin,\nUbiquitin-Like Protein...","FWP007, S863-7, CPAMD5","123,\nUniprot# 156, etc",Mandatory for all Monoclonal. Format values as...,IgG2b,"Mouse,\nHuman,\netc.",phospho-Ser301\nactetyl-Lys5\netc.,"Values may include Terminality (C-terminal, N-...",85 mg/mL by Refractometry,Recombinant corresponding to mouse 4-1BB/TNFRS...,55 kDa,1 to 125 \n(please use 'to' for indicating a r...,"Affinity purified,\nPurified by precipitation,...",CTGAACAARCTSTWCTGCCAG; CRCACACRCGCACYTYAA,Lyophilized\nliquid\nConcentrated,"F(c),\nF(ab')2",pH 6.5,Store mouse anti-GFP at -20° C prior to openin...,4°C,Antiserum; Ascites fluid; etc.,"List if available, otherwise 'No'",Yes/No,Yes/No,Yes/No,Yes/No,Yes/No,Yes/No,Yes/No,Yes/No,Yes/No,Dilute 1µl in 199µl of 5% skim milk; Do not fr...,"Dot Blot, Lateral Flow; Turbidity; etc. (300 c...",www.supplierdatapage/1234.com
1,Blue = Provide All Available\nGreen = Mandatory,Group Key,VWR US Cat. No.,VWR CA Cat. No.,VWR EU Part No.,Supplier Part No.,Title / Short Description / Antibody Name,Short Description,Long Text Description,Brand Name,Size (with unit),Packaging Format,No. of Preparations (if Kit),Type,Host,Antigen\nSymbol,Conjugation,Clonality,Antigen Name,Antigen Synonyms,Gene ID,Clone Name/Number,Isotype,Reactivity,Modification,Epitope,Concentration,Immunogen,Molecular Weight,Amino Acid Number,Purification Method,Sequence,Form,Antibody Fragment,Storage Buffer,Storage Temperature,Shipping Temperature,Format,Cross Adsorption,ELISA,Western Blot,Immunochemistry,Immunoprecipitation,Immunofluorescence,Gel Shift,CHiP,Neutralization,Flow Cytometry,Application Notes,Other Applications,URL ; Supplier Product Page for verification


In [198]:
VWREnrichmentDriver('input_sheets/NewVWRTest.xlsx', database_sheets[6], database_sheets[3], database_sheets[1], categories)
#[magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept]